# Import Libraries

In [3]:
import os
import cv2
import numpy as np
from keras.utils.np_utils import to_categorical
from keras.layers import  MaxPooling2D
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D
from keras.models import Sequential
from keras.models import model_from_json
import pickle
from sklearn.model_selection import train_test_split

# Define the path to the dataset

In [4]:
path = 'Dataset'

# Initialize lists to store image data and corresponding labels

In [5]:
X = []
Y = []

# Iterate through the dataset directory

In [6]:
for root, dirs, directory in os.walk(path):
    for j in range(len(directory)):
        name = os.path.basename(root)
        if 'Thumbs.db' not in directory[j]:
            img = cv2.imread(root+"/"+directory[j])
            img = cv2.resize(img, (64,64))
            im2arr = np.array(img)
            im2arr = im2arr.reshape(64,64,3)
            X.append(im2arr)
            lbl = 0
            if name == 'Plaque':
                lbl = 1
            Y.append(lbl)
            print(name+" "+root+"/"+directory[j]+" "+str(lbl))

Noplaque Dataset\Noplaque/aug_0_1141.png 0
Noplaque Dataset\Noplaque/aug_0_1258.png 0
Noplaque Dataset\Noplaque/aug_0_1436.png 0
Noplaque Dataset\Noplaque/aug_0_1451.png 0
Noplaque Dataset\Noplaque/aug_0_2128.png 0
Noplaque Dataset\Noplaque/aug_0_2248.png 0
Noplaque Dataset\Noplaque/aug_0_2380.png 0
Noplaque Dataset\Noplaque/aug_0_2413.png 0
Noplaque Dataset\Noplaque/aug_0_260.png 0
Noplaque Dataset\Noplaque/aug_0_3010.png 0
Noplaque Dataset\Noplaque/aug_0_3634.png 0
Noplaque Dataset\Noplaque/aug_0_3873.png 0
Noplaque Dataset\Noplaque/aug_0_4299.png 0
Noplaque Dataset\Noplaque/aug_0_439.png 0
Noplaque Dataset\Noplaque/aug_0_4480.png 0
Noplaque Dataset\Noplaque/aug_0_4710.png 0
Noplaque Dataset\Noplaque/aug_0_5226.png 0
Noplaque Dataset\Noplaque/aug_0_5301.png 0
Noplaque Dataset\Noplaque/aug_0_5456.png 0
Noplaque Dataset\Noplaque/aug_0_555.png 0
Noplaque Dataset\Noplaque/aug_0_5929.png 0
Noplaque Dataset\Noplaque/aug_0_5964.png 0
Noplaque Dataset\Noplaque/aug_0_6362.png 0
Noplaque Datas

# Convert lists to numpy arrays and Normalize the image data

In [7]:
# Convert lists to numpy arrays and Normalize the image data
X = np.asarray(X)
Y = np.asarray(Y)
X = X.astype('float32')
X = X/255

# Shuffle the dataset

In [8]:
# Shuffle the dataset
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
X = X[indices]
Y = Y[indices]

# Convert labels to categorical format

In [9]:
# Convert labels to categorical format
Y = to_categorical(Y)

# Save and Load preprocessed data

In [10]:
# Save and Load preprocessed data
np.save('model/X.txt', X)
np.save('model/Y.txt', Y)
X = np.load('model/X.txt.npy')
Y = np.load('model/Y.txt.npy')

# Split the dataset into training and testing sets

In [11]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

# Generate or Load RCNN model

In [16]:
# Load or create a RCNN model
if os.path.exists('model/model.json'):
    # Load model from JSON and weights from H5 file
    with open('model/model.json', "r") as json_file:
        loaded_model_json = json_file.read()
        classifier = model_from_json(loaded_model_json)
    json_file.close()
    classifier.load_weights("model/model_weights.h5")
else:
    # Create a new CNN model
    classifier = Sequential()
    classifier.add(Convolution2D(32, 3, 3, input_shape=(64, 64, 3), activation='relu'))
    classifier.add(MaxPooling2D(pool_size=(2, 2)))
    classifier.add(Convolution2D(32, 3, 3, activation='relu'))
    classifier.add(MaxPooling2D(pool_size=(2, 2)))
    classifier.add(Flatten())
    classifier.add(Dense(output_dim=256, activation='relu'))
    classifier.add(Dense(output_dim=Y.shape[1], activation='softmax'))


# Print Model Summary

In [17]:
 # Print model summary
print(classifier.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 31, 31, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 14, 14, 32)        0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 6272)              0         
                                                                 
 dense_1 (Dense)             (None, 256)              

# Compile the Model

In [18]:
# Compile the model
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Train the Model

In [19]:
# Train the model
hist = classifier.fit(X_train, y_train, batch_size=16, epochs=20,
                          shuffle=True, verbose=2, validation_data=(X_test,y_test))

Epoch 1/20
4/4 - 2s - loss: 0.0017 - accuracy: 1.0000 - val_loss: 5.1168e-06 - val_accuracy: 1.0000 - 2s/epoch - 501ms/step
Epoch 2/20
4/4 - 0s - loss: 0.0053 - accuracy: 1.0000 - val_loss: 0.0119 - val_accuracy: 1.0000 - 265ms/epoch - 66ms/step
Epoch 3/20
4/4 - 0s - loss: 0.0110 - accuracy: 1.0000 - val_loss: 1.9434e-04 - val_accuracy: 1.0000 - 254ms/epoch - 63ms/step
Epoch 4/20
4/4 - 0s - loss: 7.5225e-04 - accuracy: 1.0000 - val_loss: 8.6387e-04 - val_accuracy: 1.0000 - 256ms/epoch - 64ms/step
Epoch 5/20
4/4 - 0s - loss: 0.0031 - accuracy: 1.0000 - val_loss: 0.0013 - val_accuracy: 1.0000 - 252ms/epoch - 63ms/step
Epoch 6/20
4/4 - 0s - loss: 0.0034 - accuracy: 1.0000 - val_loss: 3.7805e-04 - val_accuracy: 1.0000 - 264ms/epoch - 66ms/step
Epoch 7/20
4/4 - 0s - loss: 6.9973e-04 - accuracy: 1.0000 - val_loss: 2.7307e-05 - val_accuracy: 1.0000 - 297ms/epoch - 74ms/step
Epoch 8/20
4/4 - 1s - loss: 3.3942e-05 - accuracy: 1.0000 - val_loss: 6.5701e-05 - val_accuracy: 1.0000 - 532ms/epoch - 

# Save Model weights and Architecture

In [20]:
# Save model weights and architecture
classifier.save_weights('model/model_weights.h5')
model_json = classifier.to_json()
with open("model/model.json", "w") as json_file:
    json_file.write(model_json)
json_file.close()

# Save Training History

In [21]:
# Save training history
f = open('model/history.pckl', 'wb')
pickle.dump(hist.history, f)
f.close()